In [16]:
import os

import numpy as np
from tifffile import imwrite

from deepcell_tracking.isbi_utils import trk_to_isbi
from deepcell_tracking.trk_io import load_trks

In [ ]:
data_dir = '../data'
benchmark_dir = os.path.join(data_dir, 'benchmarking_data')

Load the test split of the tracking data

In [4]:
data = load_trks('../data/test.trks')

Convert each batch of the test split to the standard ISBI format which is compatible with most of the models that we will tst.

In [17]:
for batch_no in range(len(data['lineages'])):
    # Build subdirectories for data
    raw_dir = os.path.join(benchmark_dir, '{:03}'.format(batch_no + 1))
    gt_dir = os.path.join(benchmark_dir, '{:03}_GT'.format(batch_no + 1))
    seg_dir = os.path.join(gt_dir, 'SEG')
    tra_dir = os.path.join(gt_dir, 'TRA')
    
    # Create directories if needed
    for d in (raw_dir, gt_dir, seg_dir, tra_dir):
        if not os.path.exists(d):
            os.makedirs(d)
                
    # Pull out relevant data for this batch
    x = data['X'][batch_no]
    y = data['y'][batch_no]
    lineages = data['lineages'][batch_no]
    
    # Need to translate lineages and adjust images to match restrictive ISBI format
    # Prepare output txt
    text_file = os.path.join(tra_dir, 'man_track.txt')
    df = trk_to_isbi(lineages)
    df.to_csv(text_file, sep=' ')
    
    # Determine which frames are zero padding
    frames = np.sum(y, axis=(1,2)) # True if image not blank
    good_frames = np.where(frames)[0]
    # We assume here that the empty frames are at the end of the movie (padding rather than skipped)
    movie_len = len(good_frames)
    
    # Save each frame of the movie as an individual tif
    channel = 0 # These images should only have one channel
    for i in range(movie_len):
        name_raw = os.path.join(raw_dir, 't{:03}_.tif'.format(i))
        name_tracked_seg = os.path.join(seg_dir, 'man_seg{:03}.tif'.format(i))
        name_tracked_tra = os.path.join(tra_dir, 'man_track{:03}.tif'.format(i))
        
        imwrite(name_raw, X[batch_no, i, ..., channel])
        imwrite(name_tracked_seg, y[batch_no, i, ..., channel].astype('uint16'))
        imwrite(name_tracked_tra, y[batch_no, i, ..., channel].astype('uint16'))